# A.3 - RGB+Depth (Real) 4-Channel Training

**Experiment:** A.3  
**Input:** RGB+Depth (4-channel RGBD)  
**Objective:** Test fusion of RGB and real depth data  
**Classes:** 1 (fresh_fruit_bunch)

## Workflow
1. Load RGBD dataset (4-channel images, YOLO format)
2. Train YOLOv11n with 5 seeds (42, 123, 456, 789, 101)
3. Evaluate on test set
4. Calculate mean ± std deviation

## Training Config
- Model: YOLOv11n (modified for 4-channel input)
- Epochs: 100
- Patience: 30
- Other parameters: default

## Dataset Structure
- Images: 4-channel PNG (R, G, B, Depth)
- Depth channel: normalized 0-255

In [1]:
# =============================================================================
# Cell 1: Setup + Merge RGB+Depth into 4-Channel Images
# =============================================================================
import os
import torch
import numpy as np
import cv2
import shutil
from pathlib import Path
from datetime import datetime

IS_KAGGLE = os.path.exists('/kaggle/input')
print(f"Running on: {'Kaggle' if IS_KAGGLE else 'Local'}")

# Paths
DATASET_PATH = Path('/kaggle/input/ffb-localization-rgbd-dataset/ffb_localization_rgbd')
BASE_PATH = Path('/kaggle/working')
RUNS_PATH = BASE_PATH / 'runs' / 'detect'
KAGGLE_OUTPUT = BASE_PATH / 'kaggleoutput'
KAGGLE_OUTPUT.mkdir(parents=True, exist_ok=True)

print(f"Dataset: {DATASET_PATH} (exists: {DATASET_PATH.exists()})")
print(f"CUDA: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

# =============================================================================
# Verify Original Dataset Structure (RGB, Depth, Labels)
# =============================================================================
print("\n" + "="*60)
print("ORIGINAL DATASET STRUCTURE")
print("="*60)

for split in ['train', 'val', 'test']:
    rgb_count = len(list((DATASET_PATH / 'rgb' / split).glob('*.png')))
    depth_count = len(list((DATASET_PATH / 'depth' / split).glob('*.png')))
    label_count = len(list((DATASET_PATH / 'labels' / split).glob('*.txt')))
    print(f"  {split}: {rgb_count} RGB, {depth_count} Depth, {label_count} labels")

# =============================================================================
# Merge RGB + Depth into 4-Channel RGBD Images
# =============================================================================
print("\n" + "="*60)
print("MERGING RGB + DEPTH INTO 4-CHANNEL IMAGES")
print("="*60)

def merge_rgbd(rgb_dir, depth_dir, output_dir):
    """
    Merge RGB and Depth into 4-channel RGBD images
    
    Args:
        rgb_dir: Path to RGB images folder
        depth_dir: Path to Depth images folder
        output_dir: Path to output 4-channel images folder
    
    Returns:
        Number of images merged
    """
    output_dir.mkdir(parents=True, exist_ok=True)
    
    rgb_files = sorted(rgb_dir.glob('*.png'))
    
    if len(rgb_files) == 0:
        print(f"  WARNING: No RGB images found in {rgb_dir}")
        return 0
    
    for rgb_file in rgb_files:
        # Load RGB image (BGR in OpenCV)
        rgb = cv2.imread(str(rgb_file), cv2.IMREAD_COLOR)
        if rgb is None:
            print(f"  ERROR: Cannot read {rgb_file}")
            continue
        
        # Load Depth image (single channel)
        depth_file = depth_dir / rgb_file.name
        depth = cv2.imread(str(depth_file), cv2.IMREAD_GRAYSCALE)
        if depth is None:
            print(f"  ERROR: Cannot read {depth_file}")
            continue
        
        # Merge BGR (3 channels) + Depth (1 channel) = BGRD (4 channels)
        rgbd = cv2.merge([rgb[:,:,0], rgb[:,:,1], rgb[:,:,2], depth])
        
        # Save as 4-channel PNG
        output_file = output_dir / rgb_file.name
        cv2.imwrite(str(output_file), rgbd)
    
    return len(rgb_files)

# Create merged images directory
MERGED_IMAGES_PATH = BASE_PATH / 'images_rgbd'

# Merge for train, val, test splits
merged_counts = {}
for split in ['train', 'val', 'test']:
    rgb_dir = DATASET_PATH / 'rgb' / split
    depth_dir = DATASET_PATH / 'depth' / split
    output_dir = MERGED_IMAGES_PATH / split
    
    count = merge_rgbd(rgb_dir, depth_dir, output_dir)
    merged_counts[split] = count
    print(f"  {split}: merged {count} images")

# =============================================================================
# Verify Merged 4-Channel Images
# =============================================================================
print("\n" + "="*60)
print("VERIFICATION: 4-CHANNEL MERGED IMAGES")
print("="*60)

for split in ['train', 'val', 'test']:
    img_path = MERGED_IMAGES_PATH / split
    img_files = list(img_path.glob('*.png'))
    
    if len(img_files) == 0:
        print(f"  {split}: 0 images")
        continue
    
    # Check first image
    sample_img = cv2.imread(str(img_files[0]), cv2.IMREAD_UNCHANGED)
    if sample_img is not None:
        print(f"  {split}: {len(img_files)} images, shape={sample_img.shape} (H×W×Channels)")
        assert sample_img.shape[2] == 4, f"Expected 4 channels, got {sample_img.shape[2]}"
    else:
        print(f"  {split}: Failed to read sample image")

# =============================================================================
# Copy Labels to Merged Dataset Folder
# =============================================================================
print("\n" + "="*60)
print("COPYING LABELS TO MERGED DATASET")
print("="*60)

LABELS_PATH = BASE_PATH / 'labels_rgbd'

for split in ['train', 'val', 'test']:
    src_labels_dir = DATASET_PATH / 'labels' / split
    dst_labels_dir = LABELS_PATH / split
    dst_labels_dir.mkdir(parents=True, exist_ok=True)
    
    label_files = list(src_labels_dir.glob('*.txt'))
    for label_file in label_files:
        shutil.copy(str(label_file), str(dst_labels_dir / label_file.name))
    
    print(f"  {split}: copied {len(label_files)} label files")

# =============================================================================
# Verify Final Dataset Structure
# =============================================================================
print("\n" + "="*60)
print("FINAL DATASET STRUCTURE (Ready for Training)")
print("="*60)

for split in ['train', 'val', 'test']:
    imgs = len(list((MERGED_IMAGES_PATH / split).glob('*.png')))
    lbls = len(list((LABELS_PATH / split).glob('*.txt')))
    print(f"  {split}: {imgs} images, {lbls} labels")

# =============================================================================
# Setup Paths for Training
# =============================================================================
print("\n" + "="*60)
print("TRAINING DATASET PATHS")
print("="*60)
print(f"Images root: {MERGED_IMAGES_PATH}")
print(f"Labels root: {LABELS_PATH}")
print(f"Runs output: {RUNS_PATH}")
print(f"Output directory: {KAGGLE_OUTPUT}")

print("\nSetup complete! Ready for training.")

Running on: Kaggle
Dataset: /kaggle/input/ffb-localization-rgbd-dataset/ffb_localization_rgbd (exists: True)
CUDA: True
GPU: Tesla T4

ORIGINAL DATASET STRUCTURE
  train: 280 RGB, 280 Depth, 280 labels
  val: 80 RGB, 80 Depth, 80 labels
  test: 40 RGB, 40 Depth, 40 labels

MERGING RGB + DEPTH INTO 4-CHANNEL IMAGES
  train: merged 280 images
  val: merged 80 images
  test: merged 40 images

VERIFICATION: 4-CHANNEL MERGED IMAGES
  train: 280 images, shape=(720, 1280, 4) (H×W×Channels)
  val: 80 images, shape=(720, 1280, 4) (H×W×Channels)
  test: 40 images, shape=(720, 1280, 4) (H×W×Channels)

COPYING LABELS TO MERGED DATASET
  train: copied 280 label files
  val: copied 80 label files
  test: copied 40 label files

FINAL DATASET STRUCTURE (Ready for Training)
  train: 280 images, 280 labels
  val: 80 images, 80 labels
  test: 40 images, 40 labels

TRAINING DATASET PATHS
Images root: /kaggle/working/images_rgbd
Labels root: /kaggle/working/labels_rgbd
Runs output: /kaggle/working/runs/det

In [2]:
# =============================================================================
# Cell 2: Create YAML Configuration (FIXED - SIMPLE & CLEAN)
# =============================================================================

# Buat YAML file dengan format yang benar
yaml_content = """# A.3 RGB+Depth 4-Channel Dataset
path: /kaggle/working
train: images_rgbd/train
val: images_rgbd/val
test: images_rgbd/test

nc: 1
names: ['fresh_fruit_bunch']
"""

config_path = Path('/kaggle/working/dataset_rgbd_4ch.yaml')

# Write YAML
with open(config_path, 'w') as f:
    f.write(yaml_content)

print(f"YAML Config created: {config_path}")
print(f"Exists: {config_path.exists()}\n")
print("="*60)
print("YAML CONTENT:")
print("="*60)
print(config_path.read_text())
print("="*60)

# Verify with YAML parser
import yaml

try:
    with open(config_path, 'r') as f:
        yaml_data = yaml.safe_load(f)
    print("\n✓ YAML format is valid!")
    print(f"\nParsed content:")
    for key, value in yaml_data.items():
        print(f"  {key}: {value}")
except Exception as e:
    print(f"\n✗ YAML parsing error: {e}")

print("\n✓ Ready for training!")

YAML Config created: /kaggle/working/dataset_rgbd_4ch.yaml
Exists: True

YAML CONTENT:
# A.3 RGB+Depth 4-Channel Dataset
path: /kaggle/working
train: images_rgbd/train
val: images_rgbd/val
test: images_rgbd/test

nc: 1
names: ['fresh_fruit_bunch']


✓ YAML format is valid!

Parsed content:
  path: /kaggle/working
  train: images_rgbd/train
  val: images_rgbd/val
  test: images_rgbd/test
  nc: 1
  names: ['fresh_fruit_bunch']

✓ Ready for training!


In [3]:
# =============================================================================
# Cell 3: Verify YAML (SIMPLIFIED)
# =============================================================================
config_path = Path('/kaggle/working/dataset_rgbd_4ch.yaml')
print(f"YAML: {config_path}")
print(config_path.read_text())
print("\n✓ Verification sudah dilakukan di Cell 1")
print("✓ Ready for training!")

YAML: /kaggle/working/dataset_rgbd_4ch.yaml
# A.3 RGB+Depth 4-Channel Dataset
path: /kaggle/working
train: images_rgbd/train
val: images_rgbd/val
test: images_rgbd/test

nc: 1
names: ['fresh_fruit_bunch']


✓ Verification sudah dilakukan di Cell 1
✓ Ready for training!


In [4]:
# =============================================================================
# Cell 4: Install Ultralytics
# =============================================================================
!pip install -q ultralytics

from ultralytics import YOLO
import pandas as pd
print("Ultralytics ready")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 26.0 MB/s eta 0:00:0000:01
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics ready


In [5]:
# =============================================================================
# Cell 5: Training Config
# =============================================================================
SEEDS = [42, 123, 456, 789, 101]
EXP_PREFIX = 'exp_a3_rgbd'

print(f"Seeds: {SEEDS} ({len(SEEDS)} runs)")
print("Metrics: mean ± std deviation")
print("Note: Model will be modified for 4-channel input")

Seeds: [42, 123, 456, 789, 101] (5 runs)
Metrics: mean ± std deviation
Note: Model will be modified for 4-channel input


In [6]:
# =============================================================================
# FIXED: Helper Function untuk Modify YOLO Model untuk 4-Channel Input
# =============================================================================

import torch
from ultralytics import YOLO

def modify_yolo_for_4ch(model_path='yolo11n.pt'):
    """
    Modify YOLOv11n untuk accept 4-channel input (RGBD)
    
    Default YOLO expects 3-channel (RGB) input.
    Fungsi ini memodifikasi first convolutional layer untuk accept 4 channels.
    
    Args:
        model_path: Path ke YOLO model weights
    
    Returns:
        Modified YOLO model ready untuk 4-channel input
    """
    print("Loading YOLO model...")
    model = YOLO(model_path)
    
    # DEBUG: Print model structure untuk understand
    print("\nModel structure exploration:")
    print(f"Type of model.model: {type(model.model)}")
    print(f"Type of model.model.model: {type(model.model.model)}")
    print(f"Length of model.model.model: {len(model.model.model)}")
    
    # Get first layer (should be Detect or Conv)
    first_layer = model.model.model[0]
    print(f"First layer type: {type(first_layer)}")
    print(f"First layer: {first_layer}")
    
    # Iterate untuk find first Conv layer
    first_conv = None
    for i, layer in enumerate(model.model.model):
        print(f"\nLayer {i}: {type(layer).__name__}")
        if hasattr(layer, 'conv'):
            first_conv = layer.conv
            print(f"  Found conv at layer {i}")
            break
        elif hasattr(layer, 'convs'):  # Multiple convs
            first_conv = layer.convs[0] if isinstance(layer.convs, (list, tuple)) else layer.convs
            print(f"  Found convs at layer {i}")
            break
    
    if first_conv is None:
        # Fallback: Try to get first Conv dari model
        print("\nFallback: Searching dalam model.modules()...")
        for module in model.model.modules():
            if isinstance(module, torch.nn.Conv2d):
                first_conv = module
                print(f"  Found Conv2d: in_channels={module.in_channels}, out_channels={module.out_channels}")
                break
    
    if first_conv is None:
        raise RuntimeError("Could not find Conv2d layer in model!")
    
    print(f"\n{'='*60}")
    print("MODIFYING CONV LAYER FOR 4-CHANNEL INPUT")
    print(f"{'='*60}")
    print(f"Original in_channels: {first_conv.in_channels}")
    print(f"Original weight shape: {first_conv.weight.shape}")
    print(f"  [out_channels={first_conv.weight.shape[0]}, in_channels={first_conv.weight.shape[1]}, kernel_h={first_conv.weight.shape[2]}, kernel_w={first_conv.weight.shape[3]}]")
    
    # Modifikasi in_channels dari 3 ke 4
    first_conv.in_channels = 4
    
    # Modifikasi weight tensor
    # Original: [out_channels, 3, kernel_h, kernel_w]
    # Target: [out_channels, 4, kernel_h, kernel_w]
    
    original_weight = first_conv.weight.data
    
    # Replicate first channel untuk Depth initialization
    # Ini adalah reasonable initialization karena depth channel akan di-learn
    new_weight = torch.cat([
        original_weight,
        original_weight[:, :1, :, :]  # Replicate first channel (Blue) untuk Depth
    ], dim=1)
    
    first_conv.weight = torch.nn.Parameter(new_weight)
    
    print(f"\nModified in_channels: {first_conv.in_channels}")
    print(f"Modified weight shape: {first_conv.weight.shape}")
    print(f"  [out_channels={first_conv.weight.shape[0]}, in_channels={first_conv.weight.shape[1]}, kernel_h={first_conv.weight.shape[2]}, kernel_w={first_conv.weight.shape[3]}]")
    print(f"\n✓ Model successfully modified untuk 4-channel input!")
    
    return model

# =============================================================================
# TEST: Verify modification berhasil
# =============================================================================
print("\n" + "="*60)
print("TESTING MODEL MODIFICATION")
print("="*60)

try:
    test_model = modify_yolo_for_4ch('yolo11n.pt')
    print("\n✓ Model modification successful!")
except Exception as e:
    print(f"\n✗ Error during modification: {e}")
    import traceback
    traceback.print_exc()


TESTING MODEL MODIFICATION
Loading YOLO model...

Model structure exploration:
Type of model.model: <class 'ultralytics.nn.tasks.DetectionModel'>
Type of model.model.model: <class 'torch.nn.modules.container.Sequential'>
Length of model.model.model: 24
First layer type: <class 'ultralytics.nn.modules.conv.Conv'>
First layer: Conv(
  (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (bn): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
  (act): SiLU(inplace=True)
)

Layer 0: Conv
  Found conv at layer 0

MODIFYING CONV LAYER FOR 4-CHANNEL INPUT
Original in_channels: 3
Original weight shape: torch.Size([16, 3, 3, 3])
  [out_channels=16, in_channels=3, kernel_h=3, kernel_w=3]

Modified in_channels: 4
Modified weight shape: torch.Size([16, 4, 3, 3])
  [out_channels=16, in_channels=4, kernel_h=3, kernel_w=3]

✓ Model successfully modified untuk 4-channel input!

✓ Model modification successful!


In [7]:
# =============================================================================
# NEW CELL: Fix Dataset Structure (Insert sebelum Cell 6)
# =============================================================================
# Jalankan ini SEBELUM training untuk fix label issue

import os
from pathlib import Path

print("="*60)
print("FIXING DATASET STRUCTURE FOR YOLO")
print("="*60)

BASE_PATH = Path('/kaggle/working')

# Create standard structure
IMAGES_PATH = BASE_PATH / 'images'
LABELS_PATH = BASE_PATH / 'labels'

print(f"\nCreating directory structure...")
IMAGES_PATH.mkdir(exist_ok=True)
LABELS_PATH.mkdir(exist_ok=True)

# Create symlinks untuk images dan labels
print("\nCreating symlinks...")
for split in ['train', 'val', 'test']:
    # Images
    src_img = BASE_PATH / 'images_rgbd' / split
    dst_img = IMAGES_PATH / split
    
    if src_img.exists():
        if dst_img.exists():
            if dst_img.is_symlink():
                os.unlink(dst_img)
            else:
                import shutil
                shutil.rmtree(dst_img)
        
        os.symlink(src_img, dst_img)
        print(f"  ✓ {dst_img.name} -> {src_img.name}")
    
    # Labels
    src_lbl = BASE_PATH / 'labels_rgbd' / split
    dst_lbl = LABELS_PATH / split
    
    if src_lbl.exists():
        if dst_lbl.exists():
            if dst_lbl.is_symlink():
                os.unlink(dst_lbl)
            else:
                import shutil
                shutil.rmtree(dst_lbl)
        
        os.symlink(src_lbl, dst_lbl)
        print(f"  ✓ {dst_lbl.name} -> {src_lbl.name}")

# Update YAML dengan path yang benar
print("\nUpdating YAML configuration...")

yaml_content = """path: /kaggle/working
train: images/train
val: images/val
test: images/test

nc: 1
names: ['fresh_fruit_bunch']
"""

config_path = Path('/kaggle/working/dataset_rgbd_4ch.yaml')
with open(config_path, 'w') as f:
    f.write(yaml_content)

print(f"✓ Updated: {config_path}")

# Verify structure
print("\n" + "="*60)
print("VERIFICATION")
print("="*60)

print("\nDirectory structure:")
print(f"  {IMAGES_PATH}:")
for split in ['train', 'val', 'test']:
    path = IMAGES_PATH / split
    if path.exists():
        count = len(list(path.glob('*.png')))
        print(f"    {split}: {count} images")

print(f"\n  {LABELS_PATH}:")
for split in ['train', 'val', 'test']:
    path = LABELS_PATH / split
    if path.exists():
        count = len(list(path.glob('*.txt')))
        print(f"    {split}: {count} labels")

print(f"\nYAML content:")
print(config_path.read_text())

print("="*60)
print("✓ Dataset structure is now YOLO-compatible!")
print("="*60)

FIXING DATASET STRUCTURE FOR YOLO

Creating directory structure...

Creating symlinks...
  ✓ train -> train
  ✓ train -> train
  ✓ val -> val
  ✓ val -> val
  ✓ test -> test
  ✓ test -> test

Updating YAML configuration...
✓ Updated: /kaggle/working/dataset_rgbd_4ch.yaml

VERIFICATION

Directory structure:
  /kaggle/working/images:
    train: 280 images
    val: 80 images
    test: 40 images

  /kaggle/working/labels:
    train: 280 labels
    val: 80 labels
    test: 40 labels

YAML content:
path: /kaggle/working
train: images/train
val: images/val
test: images/test

nc: 1
names: ['fresh_fruit_bunch']

✓ Dataset structure is now YOLO-compatible!


In [8]:
# =============================================================================
# Cell 6: Training Loop (4-Channel RGBD) - FIXED
# =============================================================================

print("\n" + "="*60)
print("STARTING TRAINING A.3 (4-CHANNEL RGBD)")
print("="*60)

for seed in SEEDS:
    print(f"\n{'='*60}")
    print(f"TRAINING A.3 RGBD (4-CH) - Seed {seed} ({SEEDS.index(seed)+1}/{len(SEEDS)})")
    print(f"{'='*60}\n")
    
    torch.manual_seed(seed)
    np.random.seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
    
    # Load dan modify model untuk 4-channel input
    model = modify_yolo_for_4ch('yolo11n.pt')
    
    results = model.train(
        data=str(config_path),  # config_path dari Cell 2
        epochs=1,  # PENTING: Change dari 1 ke 100 untuk production!
        patience=30,
        seed=seed,
        name=f"{EXP_PREFIX}_seed{seed}",
        exist_ok=True,
        imgsz=640,  # Resize ke 640 (dari 720)
    )
    
    print(f"\nSeed {seed} complete!")
    print(f"mAP50: {results.results_dict.get('metrics/mAP50(B)', 0):.3f}")


STARTING TRAINING A.3 (4-CHANNEL RGBD)

TRAINING A.3 RGBD (4-CH) - Seed 42 (1/5)

Loading YOLO model...

Model structure exploration:
Type of model.model: <class 'ultralytics.nn.tasks.DetectionModel'>
Type of model.model.model: <class 'torch.nn.modules.container.Sequential'>
Length of model.model.model: 24
First layer type: <class 'ultralytics.nn.modules.conv.Conv'>
First layer: Conv(
  (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (bn): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
  (act): SiLU(inplace=True)
)

Layer 0: Conv
  Found conv at layer 0

MODIFYING CONV LAYER FOR 4-CHANNEL INPUT
Original in_channels: 3
Original weight shape: torch.Size([16, 3, 3, 3])
  [out_channels=16, in_channels=3, kernel_h=3, kernel_w=3]

Modified in_channels: 4
Modified weight shape: torch.Size([16, 4, 3, 3])
  [out_channels=16, in_channels=4, kernel_h=3, kernel_w=3]

✓ Model successfully modified untuk 4-channel input!
Ultra

/usr/local/lib/python3.12/dist-packages/ultralytics/utils/metrics.py:655: RuntimeWarning: Mean of empty slice.
  ax.plot(px, py.mean(1), linewidth=3, color="blue", label=f"all classes {ap[:, 0].mean():.3f} mAP@0.5")
/usr/local/lib/python3.12/dist-packages/numpy/_core/_methods.py:138: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.12/dist-packages/ultralytics/utils/metrics.py:701: RuntimeWarning: Mean of empty slice.
  y = smooth(py.mean(0), 0.1)
/usr/local/lib/python3.12/dist-packages/numpy/_core/_methods.py:130: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(
/usr/local/lib/python3.12/dist-packages/ultralytics/utils/metrics.py:701: RuntimeWarning: Mean of empty slice.
  y = smooth(py.mean(0), 0.1)
/usr/local/lib/python3.12/dist-packages/numpy/_core/_methods.py:130: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(
/usr/local/lib/python3.12/dist-packages/ultraly

                   all         80          0          0          0          0          0
WARNING ⚠️ no labels found in detect set, cannot compute metrics without labels

1 epochs completed in 0.009 hours.
Optimizer stripped from /kaggle/working/runs/detect/exp_a3_rgbd_seed42/weights/last.pt, 5.4MB
Optimizer stripped from /kaggle/working/runs/detect/exp_a3_rgbd_seed42/weights/best.pt, 5.4MB

Validating /kaggle/working/runs/detect/exp_a3_rgbd_seed42/weights/best.pt...
Ultralytics 8.4.7 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
YOLO11n summary (fused): 101 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 3/3 5.0it/s 0.6s0.3s


/usr/local/lib/python3.12/dist-packages/ultralytics/utils/metrics.py:655: RuntimeWarning: Mean of empty slice.
  ax.plot(px, py.mean(1), linewidth=3, color="blue", label=f"all classes {ap[:, 0].mean():.3f} mAP@0.5")
/usr/local/lib/python3.12/dist-packages/numpy/_core/_methods.py:138: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.12/dist-packages/ultralytics/utils/metrics.py:701: RuntimeWarning: Mean of empty slice.
  y = smooth(py.mean(0), 0.1)
/usr/local/lib/python3.12/dist-packages/numpy/_core/_methods.py:130: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(
/usr/local/lib/python3.12/dist-packages/ultralytics/utils/metrics.py:701: RuntimeWarning: Mean of empty slice.
  y = smooth(py.mean(0), 0.1)
/usr/local/lib/python3.12/dist-packages/numpy/_core/_methods.py:130: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(
/usr/local/lib/python3.12/dist-packages/ultraly

                   all         80          0          0          0          0          0
WARNING ⚠️ no labels found in detect set, cannot compute metrics without labels
Speed: 0.1ms preprocess, 2.4ms inference, 0.0ms loss, 3.1ms postprocess per image
Results saved to /kaggle/working/runs/detect/exp_a3_rgbd_seed42

Seed 42 complete!
mAP50: 0.000

TRAINING A.3 RGBD (4-CH) - Seed 123 (2/5)

Loading YOLO model...

Model structure exploration:
Type of model.model: <class 'ultralytics.nn.tasks.DetectionModel'>
Type of model.model.model: <class 'torch.nn.modules.container.Sequential'>
Length of model.model.model: 24
First layer type: <class 'ultralytics.nn.modules.conv.Conv'>
First layer: Conv(
  (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (bn): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
  (act): SiLU(inplace=True)
)

Layer 0: Conv
  Found conv at layer 0

MODIFYING CONV LAYER FOR 4-CHANNEL INPUT
Original in_chan

...

                   all         80          0          0          0          0          0
WARNING ⚠️ no labels found in detect set, cannot compute metrics without labels
Speed: 0.1ms preprocess, 2.6ms inference, 0.0ms loss, 3.2ms postprocess per image
Results saved to /kaggle/working/runs/detect/exp_a3_rgbd_seed789

Seed 789 complete!
mAP50: 0.000

TRAINING A.3 RGBD (4-CH) - Seed 101 (5/5)

Loading YOLO model...

Model structure exploration:
Type of model.model: <class 'ultralytics.nn.tasks.DetectionModel'>
Type of model.model.model: <class 'torch.nn.modules.container.Sequential'>
Length of model.model.model: 24
First layer type: <class 'ultralytics.nn.modules.conv.Conv'>
First layer: Conv(
  (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (bn): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
  (act): SiLU(inplace=True)
)

Layer 0: Conv
  Found conv at layer 0

MODIFYING CONV LAYER FOR 4-CHANNEL INPUT
Original in_ch

/usr/local/lib/python3.12/dist-packages/ultralytics/utils/metrics.py:655: RuntimeWarning: Mean of empty slice.
  ax.plot(px, py.mean(1), linewidth=3, color="blue", label=f"all classes {ap[:, 0].mean():.3f} mAP@0.5")
/usr/local/lib/python3.12/dist-packages/numpy/_core/_methods.py:138: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.12/dist-packages/ultralytics/utils/metrics.py:701: RuntimeWarning: Mean of empty slice.
  y = smooth(py.mean(0), 0.1)
/usr/local/lib/python3.12/dist-packages/numpy/_core/_methods.py:130: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(
/usr/local/lib/python3.12/dist-packages/ultralytics/utils/metrics.py:701: RuntimeWarning: Mean of empty slice.
  y = smooth(py.mean(0), 0.1)
/usr/local/lib/python3.12/dist-packages/numpy/_core/_methods.py:130: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(
/usr/local/lib/python3.12/dist-packages/ultraly

                   all         80          0          0          0          0          0
WARNING ⚠️ no labels found in detect set, cannot compute metrics without labels

1 epochs completed in 0.004 hours.
Optimizer stripped from /kaggle/working/runs/detect/exp_a3_rgbd_seed101/weights/last.pt, 5.4MB
Optimizer stripped from /kaggle/working/runs/detect/exp_a3_rgbd_seed101/weights/best.pt, 5.4MB

Validating /kaggle/working/runs/detect/exp_a3_rgbd_seed101/weights/best.pt...
Ultralytics 8.4.7 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
YOLO11n summary (fused): 101 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 3/3 5.5it/s 0.5s0.3s


/usr/local/lib/python3.12/dist-packages/ultralytics/utils/metrics.py:655: RuntimeWarning: Mean of empty slice.
  ax.plot(px, py.mean(1), linewidth=3, color="blue", label=f"all classes {ap[:, 0].mean():.3f} mAP@0.5")
/usr/local/lib/python3.12/dist-packages/numpy/_core/_methods.py:138: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.12/dist-packages/ultralytics/utils/metrics.py:701: RuntimeWarning: Mean of empty slice.
  y = smooth(py.mean(0), 0.1)
/usr/local/lib/python3.12/dist-packages/numpy/_core/_methods.py:130: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(
/usr/local/lib/python3.12/dist-packages/ultralytics/utils/metrics.py:701: RuntimeWarning: Mean of empty slice.
  y = smooth(py.mean(0), 0.1)
/usr/local/lib/python3.12/dist-packages/numpy/_core/_methods.py:130: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(
/usr/local/lib/python3.12/dist-packages/ultraly

                   all         80          0          0          0          0          0
WARNING ⚠️ no labels found in detect set, cannot compute metrics without labels
Speed: 0.2ms preprocess, 2.0ms inference, 0.0ms loss, 2.6ms postprocess per image
Results saved to /kaggle/working/runs/detect/exp_a3_rgbd_seed101

Seed 101 complete!
mAP50: 0.000


In [9]:
# =============================================================================
# Cell 7: Evaluation on Test Set
# =============================================================================
results_dict = {}

print("="*60)
print("EVALUATION ON TEST SET")
print("="*60)

for seed in SEEDS:
    model_path = RUNS_PATH / f"{EXP_PREFIX}_seed{seed}" / 'weights' / 'best.pt'
    
    if not model_path.exists():
        print(f"Not found: {model_path}")
        continue
    
    print(f"\nSeed {seed}:")
    model = YOLO(str(model_path))
    metrics = model.val(data=str(config_path), split='test')
    
    results_dict[seed] = {
        'mAP50': metrics.box.map50,
        'mAP50-95': metrics.box.map,
        'Precision': metrics.box.mp,
        'Recall': metrics.box.mr
    }
    
    print(f"  mAP50: {metrics.box.map50:.3f}")
    print(f"  mAP50-95: {metrics.box.map:.3f}")
    print(f"  Precision: {metrics.box.mp:.3f}")
    print(f"  Recall: {metrics.box.mr:.3f}")

EVALUATION ON TEST SET

Seed 42:
Ultralytics 8.4.7 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
YOLO11n summary (fused): 101 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 3883.0±1620.0 MB/s, size: 2217.6 KB)
val: Scanning /kaggle/working/images_rgbd/test... 0 images, 40 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 40/40 86.8it/s 0.5s0.1s
WARNING ⚠️ val: No labels found in /kaggle/working/images_rgbd/test.cache. See https://docs.ultralytics.com/datasets for dataset formatting guidance.
val: New cache created: /kaggle/working/images_rgbd/test.cache
WARNING ⚠️ Labels are missing or empty in /kaggle/working/images_rgbd/test.cache, training may not work correctly. See https://docs.ultralytics.com/datasets for dataset formatting guidance.
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 3/3 2.5it/s 1.2s0.7s


/usr/local/lib/python3.12/dist-packages/ultralytics/utils/metrics.py:655: RuntimeWarning: Mean of empty slice.
  ax.plot(px, py.mean(1), linewidth=3, color="blue", label=f"all classes {ap[:, 0].mean():.3f} mAP@0.5")
/usr/local/lib/python3.12/dist-packages/numpy/_core/_methods.py:138: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.12/dist-packages/ultralytics/utils/metrics.py:701: RuntimeWarning: Mean of empty slice.
  y = smooth(py.mean(0), 0.1)
/usr/local/lib/python3.12/dist-packages/numpy/_core/_methods.py:130: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(
/usr/local/lib/python3.12/dist-packages/ultralytics/utils/metrics.py:701: RuntimeWarning: Mean of empty slice.
  y = smooth(py.mean(0), 0.1)
/usr/local/lib/python3.12/dist-packages/numpy/_core/_methods.py:130: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(
/usr/local/lib/python3.12/dist-packages/ultraly

                   all         40          0          0          0          0          0
WARNING ⚠️ no labels found in detect set, cannot compute metrics without labels
Speed: 3.7ms preprocess, 8.0ms inference, 0.0ms loss, 1.5ms postprocess per image
Results saved to /kaggle/working/runs/detect/val
  mAP50: 0.000
  mAP50-95: 0.000
  Precision: 0.000
  Recall: 0.000

Seed 123:
Ultralytics 8.4.7 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
YOLO11n summary (fused): 101 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 4096.4±1190.5 MB/s, size: 2250.6 KB)
val: Scanning /kaggle/working/images_rgbd/test.cache... 0 images, 40 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 40/40 18.6Mit/s 0.0s
WARNING ⚠️ Labels are missing or empty in /kaggle/working/images_rgbd/test.cache, training may not work correctly. See https://docs.ultralytics.com/datasets for dataset formatting guidance.
                 Class     Images  Instances    

/usr/local/lib/python3.12/dist-packages/ultralytics/utils/metrics.py:655: RuntimeWarning: Mean of empty slice.
  ax.plot(px, py.mean(1), linewidth=3, color="blue", label=f"all classes {ap[:, 0].mean():.3f} mAP@0.5")
/usr/local/lib/python3.12/dist-packages/numpy/_core/_methods.py:138: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.12/dist-packages/ultralytics/utils/metrics.py:701: RuntimeWarning: Mean of empty slice.
  y = smooth(py.mean(0), 0.1)
/usr/local/lib/python3.12/dist-packages/numpy/_core/_methods.py:130: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(
/usr/local/lib/python3.12/dist-packages/ultralytics/utils/metrics.py:701: RuntimeWarning: Mean of empty slice.
  y = smooth(py.mean(0), 0.1)
/usr/local/lib/python3.12/dist-packages/numpy/_core/_methods.py:130: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(
/usr/local/lib/python3.12/dist-packages/ultraly

                   all         40          0          0          0          0          0
WARNING ⚠️ no labels found in detect set, cannot compute metrics without labels
Speed: 2.0ms preprocess, 5.3ms inference, 0.0ms loss, 2.8ms postprocess per image
Results saved to /kaggle/working/runs/detect/val2
  mAP50: 0.000
  mAP50-95: 0.000
  Precision: 0.000
  Recall: 0.000

Seed 456:
Ultralytics 8.4.7 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
YOLO11n summary (fused): 101 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 5004.2±996.9 MB/s, size: 2183.9 KB)
val: Scanning /kaggle/working/images_rgbd/test.cache... 0 images, 40 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 40/40 16.8Mit/s 0.0s
WARNING ⚠️ Labels are missing or empty in /kaggle/working/images_rgbd/test.cache, training may not work correctly. See https://docs.ultralytics.com/datasets for dataset formatting guidance.
                 Class     Images  Instances    

...

                   all         40          0          0          0          0          0
WARNING ⚠️ no labels found in detect set, cannot compute metrics without labels
Speed: 3.4ms preprocess, 3.3ms inference, 0.0ms loss, 1.7ms postprocess per image
Results saved to /kaggle/working/runs/detect/val3
  mAP50: 0.000
  mAP50-95: 0.000
  Precision: 0.000
  Recall: 0.000

Seed 789:
Ultralytics 8.4.7 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
YOLO11n summary (fused): 101 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 5373.6±1304.2 MB/s, size: 2297.8 KB)
val: Scanning /kaggle/working/images_rgbd/test.cache... 0 images, 40 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 40/40 21.0Mit/s 0.0s
WARNING ⚠️ Labels are missing or empty in /kaggle/working/images_rgbd/test.cache, training may not work correctly. See https://docs.ultralytics.com/datasets for dataset formatting guidance.
                 Class     Images  Instances   

/usr/local/lib/python3.12/dist-packages/ultralytics/utils/metrics.py:655: RuntimeWarning: Mean of empty slice.
  ax.plot(px, py.mean(1), linewidth=3, color="blue", label=f"all classes {ap[:, 0].mean():.3f} mAP@0.5")
/usr/local/lib/python3.12/dist-packages/numpy/_core/_methods.py:138: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.12/dist-packages/ultralytics/utils/metrics.py:701: RuntimeWarning: Mean of empty slice.
  y = smooth(py.mean(0), 0.1)
/usr/local/lib/python3.12/dist-packages/numpy/_core/_methods.py:130: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(
/usr/local/lib/python3.12/dist-packages/ultralytics/utils/metrics.py:701: RuntimeWarning: Mean of empty slice.
  y = smooth(py.mean(0), 0.1)
/usr/local/lib/python3.12/dist-packages/numpy/_core/_methods.py:130: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(
/usr/local/lib/python3.12/dist-packages/ultraly

                   all         40          0          0          0          0          0
WARNING ⚠️ no labels found in detect set, cannot compute metrics without labels
Speed: 3.5ms preprocess, 3.5ms inference, 0.0ms loss, 1.8ms postprocess per image
Results saved to /kaggle/working/runs/detect/val4
  mAP50: 0.000
  mAP50-95: 0.000
  Precision: 0.000
  Recall: 0.000

Seed 101:
Ultralytics 8.4.7 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
YOLO11n summary (fused): 101 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 4490.9±1009.2 MB/s, size: 2288.2 KB)
val: Scanning /kaggle/working/images_rgbd/test.cache... 0 images, 40 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 40/40 15.3Mit/s 0.0s
WARNING ⚠️ Labels are missing or empty in /kaggle/working/images_rgbd/test.cache, training may not work correctly. See https://docs.ultralytics.com/datasets for dataset formatting guidance.
                 Class     Images  Instances   

/usr/local/lib/python3.12/dist-packages/ultralytics/utils/metrics.py:655: RuntimeWarning: Mean of empty slice.
  ax.plot(px, py.mean(1), linewidth=3, color="blue", label=f"all classes {ap[:, 0].mean():.3f} mAP@0.5")
/usr/local/lib/python3.12/dist-packages/numpy/_core/_methods.py:138: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.12/dist-packages/ultralytics/utils/metrics.py:701: RuntimeWarning: Mean of empty slice.
  y = smooth(py.mean(0), 0.1)
/usr/local/lib/python3.12/dist-packages/numpy/_core/_methods.py:130: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(
/usr/local/lib/python3.12/dist-packages/ultralytics/utils/metrics.py:701: RuntimeWarning: Mean of empty slice.
  y = smooth(py.mean(0), 0.1)
/usr/local/lib/python3.12/dist-packages/numpy/_core/_methods.py:130: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(
/usr/local/lib/python3.12/dist-packages/ultraly

                   all         40          0          0          0          0          0
WARNING ⚠️ no labels found in detect set, cannot compute metrics without labels
Speed: 3.2ms preprocess, 4.1ms inference, 0.0ms loss, 2.7ms postprocess per image
Results saved to /kaggle/working/runs/detect/val5
  mAP50: 0.000
  mAP50-95: 0.000
  Precision: 0.000
  Recall: 0.000


In [10]:
# =============================================================================
# Cell 8: Results Summary
# =============================================================================
df = pd.DataFrame(results_dict).T
df.index.name = 'Seed'

# Calculate mean and std
avg = df.mean()
std = df.std()

print("\n" + "="*60)
print("A.3 RGB+DEPTH (REAL) - FINAL RESULTS")
print("="*60 + "\n")
print(df.to_string(float_format=lambda x: f"{x:.3f}"))

print("\n" + "-"*60)
print("SUMMARY (Mean ± Std)")
print("-"*60)
for col in df.columns:
    print(f"  {col}: {avg[col]:.3f} ± {std[col]:.3f}")


A.3 RGB+DEPTH (REAL) - FINAL RESULTS

      mAP50  mAP50-95  Precision  Recall
Seed                                    
42    0.000     0.000      0.000   0.000
123   0.000     0.000      0.000   0.000
456   0.000     0.000      0.000   0.000
789   0.000     0.000      0.000   0.000
101   0.000     0.000      0.000   0.000

------------------------------------------------------------
SUMMARY (Mean ± Std)
------------------------------------------------------------
  mAP50: 0.000 ± 0.000
  mAP50-95: 0.000 ± 0.000
  Precision: 0.000 ± 0.000
  Recall: 0.000 ± 0.000


In [11]:
# =============================================================================
# Cell 9: Save Results
# =============================================================================
output_file = KAGGLE_OUTPUT / 'a3_rgbd_results.txt'

with open(output_file, 'w') as f:
    f.write("="*60 + "\n")
    f.write("A.3 RGB+Depth (Real) 4-Channel Results\n")
    f.write(f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
    f.write("Training: epochs=100, patience=30, other=default\n")
    f.write(f"Seeds: {SEEDS}\n")
    f.write("="*60 + "\n\n")
    f.write("Per-Seed Results:\n")
    f.write(df.to_string(float_format=lambda x: f"{x:.3f}"))
    f.write("\n\n" + "-"*60 + "\n")
    f.write("Summary (Mean ± Std):\n")
    for col in df.columns:
        f.write(f"  {col}: {avg[col]:.3f} ± {std[col]:.3f}\n")

print(f"Results saved: {output_file}")

Results saved: /kaggle/working/kaggleoutput/a3_rgbd_results.txt


In [12]:
# =============================================================================
# Cell 10: Create Archives
# =============================================================================
if RUNS_PATH.exists():
    shutil.make_archive('/kaggle/working/a3_runs', 'zip', RUNS_PATH)
    print(f"a3_runs.zip: {os.path.getsize('/kaggle/working/a3_runs.zip')/1024/1024:.1f} MB")

shutil.make_archive('/kaggle/working/a3_output', 'zip', KAGGLE_OUTPUT)
print("a3_output.zip created")

print("\nDownload from Output tab")

a3_runs.zip: 97.4 MB
a3_output.zip created

Download from Output tab
